In [19]:
import re
import itertools
from dataclasses import dataclass
from more_itertools import chunked

In [21]:
@dataclass
class Pattern:
    name: str
    pattern: bytes

In [29]:
special_symbols = b".()[]?\\"

In [30]:
def escape_special_symbols(char: bytes):
    if char in special_symbols:
        return b"\\" + char
    return char

In [34]:
def convert_to_pattern(s: list[str]):
    pattern = b"".join(b"." if item == "??" else escape_special_symbols(int(item, 16).to_bytes(1, "little")) for item in s)
    return pattern

In [35]:
patterns = []

with open("fn_byte_patterns.ffsess") as patterns_file:
    for tab_line, pattern_line in chunked(patterns_file, 2):
        command, _, tab_name = tab_line.rstrip().partition(" ")
        assert command == "Tab", command
        rule_name, _, _, *pattern = pattern_line.rstrip().split(" ")
        assert rule_name == "RuleBytePattern"
        patterns.append(Pattern(tab_name, convert_to_pattern(pattern)))

In [36]:
patterns

[Pattern(name='string_copy', pattern=b'H\x89Q.H\x83y..r.H\x8b\x01\xc6\x04\x10.\xc3\xc6\x04\x11.\xc3'),
 Pattern(name='string_copy_n', pattern=b'H\x89\\\\$.H\x89l$.VWAWH\x83\xec.H\x8bi.I\x8b\xf0L\x8b\xfaH\x8b\xd9L;\xc5w.H\x8b\xf9H\x83\xfd.r.H\x8b9H\x89q.H\x8b\xcf\xe8....\xc6\x047.\xe9....H\xbf........H;\xf7\x0f\x87....H\x8b\xceL\x89t$.H\x83\xc9.H;\xcfw.H\x8b\xd5H\x8b\xc7H\xd1\xeaH+\xc2H;\xe8w.H\x8d\x04*H\x8b\xf9H;\xc8H\x0fB\xf8H\x8dO.\xe8....L\x8b\xc6H\x89s.I\x8b\xd7H\x89{.H\x8b\xc8L\x8b\xf0\xe8....A\xc6\x046.H\x83\xfd.r.H\x8b\x0bH\x8dU.H\x81\xfa....r.L\x8bA.H\x83\xc2.I+\xc8H\x8dA.H\x83\xf8.w.I\x8b\xc8\xe8....L\x893L\x8bt$.H\x8bl$.H\x8b\xc3H\x8b\\\\$.H\x83\xc4.A__^\xc3\xff\x15....\xcc\xe8....\xcc'),
 Pattern(name='string_append', pattern=b'I\xc7\xc0....f\x0f\x1f\x84\x00....I\xff\xc0B\x80<\x02.u.\xe9....'),
 Pattern(name='string_append_0', pattern=b'I\xc7\xc0....f\x0f\x1f\x84\x00....I\xff\xc0B\x80<\x02.u.\xe9....'),
 Pattern(name='string_append_n', pattern=b'H\x89\\\\$.UVWAUAVH\x83\xec.H

In [37]:
patterns[2].pattern == patterns[3].pattern

True

In [40]:
path = "/mnt/second/SteamLibrary/steamapps/common/Dwarf Fortress/Dwarf Fortress.exe"

In [39]:
with open(path, "rb") as file:
    data = file.read()
    for pattern in patterns:
        for match in re.finditer(pattern.pattern, data):
            print(pattern.name, hex(match.start(0)))

string_copy 0xac30
string_append 0xa9f0
string_append 0xaa10
string_append_0 0xa9f0
string_append_0 0xaa10
addst_template 0x1199
addst_template 0x11da
addst_template 0x1b49
addst_template 0xf5c0
addst_template 0x124f9
addst_template 0x138d9
addst_template 0x1eaec
addst_template 0x1ebc6
addst_template 0x200af
addst_template 0x200c0
addst_template 0x207ab
addst_template 0x36375
addst_template 0x3b95b
addst_template 0x42a9f
addst_template 0x42c71
addst_template 0x4359b
addst_template 0x435b6
addst_template 0x435e4
addst_template 0x4392c
addst_template 0x43947
addst_template 0x43baf
addst_template 0x4405e
addst_template 0x44468
addst_template 0x44a3f
addst_template 0x44a5e
addst_template 0x45028
addst_template 0x47780
addst_template 0x4dcf7
addst_template 0x4ea11
addst_template 0x4fecb
addst_template 0x50855
addst_template 0x5105f
addst_template 0x53143
addst_template 0x54dd7
addst_template 0x556e5
addst_template 0x566ab
addst_template 0x5732a
addst_template 0x58e9c
addst_template 0x58ebb


addst_template 0x411898
addst_template 0x414bc0
addst_template 0x414f60
addst_template 0x415341
addst_template 0x415349
addst_template 0x4157c2
addst_template 0x415b60
addst_template 0x415eb7
addst_template 0x4162e8
addst_template 0x416384
addst_template 0x416830
addst_template 0x4168c4
addst_template 0x416980
addst_template 0x417080
addst_template 0x417532
addst_template 0x417882
addst_template 0x417bc2
addst_template 0x417ed2
addst_template 0x4181b2
addst_template 0x4184d2
addst_template 0x418872
addst_template 0x418bf2
addst_template 0x418f52
addst_template 0x41984d
addst_template 0x41e07f
addst_template 0x41e44f
addst_template 0x41f6a8
addst_template 0x41f8ac
addst_template 0x41fee6
addst_template 0x420211
addst_template 0x420491
addst_template 0x420704
addst_template 0x420722
addst_template 0x4207ef
addst_template 0x4208fe
addst_template 0x420917
addst_template 0x420932
addst_template 0x420b4b
addst_template 0x420d9b
addst_template 0x420dbe
addst_template 0x420e53
addst_template 0

addst_template 0x766dfd
addst_template 0x766fbe
addst_template 0x768879
addst_template 0x7688b9
addst_template 0x768969
addst_template 0x7691b9
addst_template 0x7693f9
addst_template 0x769419
addst_template 0x769439
addst_template 0x7694b0
addst_template 0x7694c0
addst_template 0x769628
addst_template 0x769a09
addst_template 0x769a40
addst_template 0x769a59
addst_template 0x769d78
addst_template 0x769db9
addst_template 0x769f99
addst_template 0x76a4ad
addst_template 0x76b1fe
addst_template 0x76b25f
addst_template 0x76e14f
addst_template 0x77156f
addst_template 0x779c1b
addst_template 0x77b03a
addst_template 0x77ba00
addst_template 0x77cca6
addst_template 0x781c83
addst_template 0x781cef
addst_template 0x7820fe
addst_template 0x78af1f
addst_template 0x790760
addst_template 0x791493
addst_template 0x79464c
addst_template 0x794dd3
addst_template 0x7a2c35
addst_template 0x7a2caa
addst_template 0x7a2d25
addst_template 0x7a2dd5
addst_template 0x7a2e4a
addst_template 0x7a2ec5
addst_template 0

addst_template 0xbcf8af
addst_template 0xbcf8c2
addst_template 0xbcf8d5
addst_template 0xbcf8e8
addst_template 0xbcf8fb
addst_template 0xbcf90e
addst_template 0xbcf921
addst_template 0xbcf934
addst_template 0xbcf947
addst_template 0xbcf95a
addst_template 0xbcf96d
addst_template 0xbcf99c
addst_template 0xbcfaca
addst_template 0xbcfadd
addst_template 0xbcfaf6
addst_template 0xbcfb09
addst_template 0xbcfb7a
addst_template 0xbcfb9a
addst_template 0xbcfbad
addst_template 0xbd011d
addst_template 0xbd0368
addst_template 0xbd0a55
addst_template 0xbd0a6a
addst_template 0xbd0af0
addst_template 0xbd0b04
addst_template 0xbd3fbc
addst_template 0xbd3fdb
addst_template 0xbd43e1
addst_template 0xbd646f
addst_template 0xbd671d
addst_template 0xbd757e
addst_template 0xbd775e
addst_template 0xbd7824
addst_template 0xbd865d
addst_template 0xbd8e81
addst_template 0xbdd2d5
addst_template 0xbdd375
addst_template 0xbdd971
addst_template 0xbde0cd
addst_template 0xbe38ad
addst_template 0xbe3b7a
addst_template 0

addst_template 0xdba442
addst_template 0xdba7e0
addst_template 0xdbab7e
addst_template 0xdbaf1c
addst_template 0xdbb2ba
addst_template 0xdbb658
addst_template 0xdbb9f6
addst_template 0xdbbd94
addst_template 0xdbc132
addst_template 0xdbc4d0
addst_template 0xdbc86e
addst_template 0xdbcc0c
addst_template 0xdbcfaa
addst_template 0xdbd348
addst_template 0xdbd6e6
addst_template 0xdbda84
addst_template 0xdbde22
addst_template 0xdbe1cb
addst_template 0xdbe586
addst_template 0xdbe941
addst_template 0xdbecfc
addst_template 0xdbf0b7
addst_template 0xdbf472
addst_template 0xdbf82d
addst_template 0xdbf94e
addst_template 0xdbf9d7
addst_template 0xdbfbfa
addst_template 0xdbfe78
addst_template 0xdc00f6
addst_template 0xdc0374
addst_template 0xdc05f2
addst_template 0xdc0870
addst_template 0xdc0aee
addst_template 0xdc0d6c
addst_template 0xdc0e50
addst_template 0xdc0ed9
addst_template 0xdc0f62
addst_template 0xdc0feb
addst_template 0xdc1074
addst_template 0xdc10fd
addst_template 0xdc1186
addst_template 0

addst_template 0xe1eebe
addst_template 0xe1ef47
addst_template 0xe1efd0
addst_template 0xe1f059
addst_template 0xe1f0e2
addst_template 0xe1f16b
addst_template 0xe1f1f4
addst_template 0xe1f27d
addst_template 0xe1f306
addst_template 0xe1f38f
addst_template 0xe1f418
addst_template 0xe1f4a1
addst_template 0xe1f52a
addst_template 0xe1f5b3
addst_template 0xe1f63c
addst_template 0xe1f6c5
addst_template 0xe1f74e
addst_template 0xe1f7d7
addst_template 0xe1f860
addst_template 0xe1f8e9
addst_template 0xe1f972
addst_template 0xe1f9fb
addst_template 0xe1fa84
addst_template 0xe1fb0d
addst_template 0xe1fb96
addst_template 0xe1fc1f
addst_template 0xe1fca8
addst_template 0xe1fd31
addst_template 0xe1fdba
addst_template 0xe1fe43
addst_template 0xe1fecc
addst_template 0xe1ff55
addst_template 0xe1ffde
addst_template 0xe20067
addst_template 0xe200f0
addst_template 0xe20179
addst_template 0xe20202
addst_template 0xe2028b
addst_template 0xe20314
addst_template 0xe2039d
addst_template 0xe20426
addst_template 0

addst_template 0xe989a2
addst_template 0xe98b1e
addst_template 0xe98c9a
addst_template 0xe98e16
addst_template 0xe98f92
addst_template 0xe9910e
addst_template 0xe9928a
addst_template 0xe99406
addst_template 0xe99582
addst_template 0xe996fe
addst_template 0xe9987a
addst_template 0xe999f6
addst_template 0xe99b72
addst_template 0xe99cee
addst_template 0xe99e6a
addst_template 0xe99fe6
addst_template 0xe9a162
addst_template 0xe9a2de
addst_template 0xe9a45a
addst_template 0xe9a5d6
addst_template 0xe9a752
addst_template 0xe9a8ce
addst_template 0xe9aa4a
addst_template 0xe9abc6
addst_template 0xe9ad42
addst_template 0xe9aebe
addst_template 0xe9b03a
addst_template 0xe9b1b6
addst_template 0xe9b332
addst_template 0xe9b4ae
addst_template 0xe9b62a
addst_template 0xe9b7a6
addst_template 0xe9b922
addst_template 0xe9ba9e
addst_template 0xe9bc1a
addst_template 0xe9bd96
addst_template 0xe9bf12
addst_template 0xe9c08e
addst_template 0xe9c20a
addst_template 0xe9c386
addst_template 0xe9c502
addst_template 0

addst_template 0x10dffb7
addst_template 0x10dffc7
addst_template 0x10e0040
addst_template 0x10e00c0
addst_template 0x10e0140
addst_template 0x10e01b7
addst_template 0x10e01c7
addst_template 0x10e01d7
addst_template 0x10e01f3
addst_template 0x10e0283
addst_template 0x10e0303
addst_template 0x10e0433
addst_template 0x10e05c3
addst_template 0x10e0673
addst_template 0x10e0703
addst_template 0x10e0793
addst_template 0x10e08e3
addst_template 0x10e1007
addst_template 0x10e1273
addst_template 0x10e1283
addst_template 0x10e1293
addst_template 0x10e132f
addst_template 0x10e1466
addst_template 0x10e14d0
addst_template 0x10e14e0
addst_template 0x10e14f0
addst_template 0x10e1530
addst_template 0x10e1540
addst_template 0x10e1550
addst_template 0x10e1560
addst_template 0x10e2e03
addst_template 0x10e3683
addst_template 0x10e3693
addst_template 0x10e36a3
addst_template 0x10e3f72
addst_template 0x10e4157
addst_template 0x10e4184
addst_template 0x10e42e3
addst_template 0x10e42f3
addst_template 0x10e4303
